In [2]:

import tensorflow as tf
import matplotlib.image as img
%matplotlib inline
import numpy as np
from collections import defaultdict
import collections
from shutil import copy
from shutil import copytree, rmtree
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import applications, optimizers
from tensorflow.keras.models import Model
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16,preprocess_input
from tensorboard.plugins.hparams import api as hp
import matplotlib as mpl
import matplotlib.pyplot as plt
import coremltools as ct 

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

np.set_printoptions(precision=3, suppress=True)

/Users/jduck/Desktop/SWENG894/CNN_Model/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
TensorFlow version 2.18.0 has not been tested with coremltools. You may run into unexpected errors. TensorFlow 2.12.0 is the most recent version that has been tested.


In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


Extract the Data

In [3]:
def get_data_extract():
  if "food-101" in os.listdir():
    print("Dataset already exists")
  else:
    print("Downloading the data...")
    !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
    print("Dataset downloaded!")
    print("Extracting data..")
    !tar xzvf food-101.tar.gz
    print("Extraction done!")

Verify that the images are labeled

In [4]:
path = './food-101/images'
os.listdir(path)

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheesecake',
 'cheese_plate',
 'chicken_curry',
 'chicken_quesadilla',
 'chicken_wings',
 'chocolate_cake',
 'chocolate_mousse',
 'churros',
 'clam_chowder',
 'club_sandwich',
 'crab_cakes',
 'creme_brulee',
 'croque_madame',
 'cup_cakes',
 'deviled_eggs',
 'donuts',
 'dumplings',
 'edamame',
 'eggs_benedict',
 'escargots',
 'falafel',
 'filet_mignon',
 'fish_and_chips',
 'foie_gras',
 'french_fries',
 'french_onion_soup',
 'french_toast',
 'fried_calamari',
 'fried_rice',
 'frozen_yogurt',
 'garlic_bread',
 'gnocchi',
 'greek_salad',
 'grilled_cheese_sandwich',
 'grilled_salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot_and_sour_soup',
 'hot_dog',
 'huevos_rancheros',
 'hummus',
 'ice_cream',
 'lasagna',
 'lobster_bisque',
 'lobster

Check the metadata of the dataset

In [5]:
os.listdir("./food-101/meta")

['classes.txt',
 'labels.txt',
 'test.json',
 'test.txt',
 'train.json',
 'train.txt']

Use a helper function provided from kaggle in the case that smaller subsets could be used to train and test.

In [3]:
# Helper method to split dataset into train and test folders
def prepare_data(filepath, src,dest):
  classes_images = defaultdict(list)
  with open(filepath, 'r') as txt:
      paths = [read.strip() for read in txt.readlines()]
      for p in paths:
        food = p.split('/')
        classes_images[food[0]].append(food[1] + '.jpg')

  for food in classes_images.keys():
    print("\nCopying images into ",food)
    if not os.path.exists(os.path.join(dest,food)):
      os.makedirs(os.path.join(dest,food))
    for i in classes_images[food]:
      copy(os.path.join(src,food,i), os.path.join(dest,food,i))
  print("Copying Done!")

In [7]:
print("Creating train data...")
prepare_data('./food-101/meta/train.txt', './food-101/images', 'train')

Creating train data...

Copying images into  apple_pie

Copying images into  baby_back_ribs

Copying images into  baklava

Copying images into  beef_carpaccio

Copying images into  beef_tartare

Copying images into  beet_salad

Copying images into  beignets

Copying images into  bibimbap

Copying images into  bread_pudding

Copying images into  breakfast_burrito

Copying images into  bruschetta

Copying images into  caesar_salad

Copying images into  cannoli

Copying images into  caprese_salad

Copying images into  carrot_cake

Copying images into  ceviche

Copying images into  cheesecake

Copying images into  cheese_plate

Copying images into  chicken_curry

Copying images into  chicken_quesadilla

Copying images into  chicken_wings

Copying images into  chocolate_cake

Copying images into  chocolate_mousse

Copying images into  churros

Copying images into  clam_chowder

Copying images into  club_sandwich

Copying images into  crab_cakes

Copying images into  creme_brulee

Copying im

In [8]:
print("Creating test data...")
prepare_data('./food-101/meta/test.txt', './food-101/images', 'test')

Creating test data...

Copying images into  apple_pie

Copying images into  baby_back_ribs

Copying images into  baklava

Copying images into  beef_carpaccio

Copying images into  beef_tartare

Copying images into  beet_salad

Copying images into  beignets

Copying images into  bibimbap

Copying images into  bread_pudding

Copying images into  breakfast_burrito

Copying images into  bruschetta

Copying images into  caesar_salad

Copying images into  cannoli

Copying images into  caprese_salad

Copying images into  carrot_cake

Copying images into  ceviche

Copying images into  cheesecake

Copying images into  cheese_plate

Copying images into  chicken_curry

Copying images into  chicken_quesadilla

Copying images into  chicken_wings

Copying images into  chocolate_cake

Copying images into  chocolate_mousse

Copying images into  churros

Copying images into  clam_chowder

Copying images into  club_sandwich

Copying images into  crab_cakes

Copying images into  creme_brulee

Copying ima

In [10]:

# Helper method to create train_mini and test_mini data samples
def dataset_mini(food_list, src, dest):
  if os.path.exists(dest):
    rmtree(dest) # removing dataset_mini(if it already exists) folders so that we will have only the classes that we want
  os.makedirs(dest)
  for food_item in food_list :
    print("Copying images into",food_item)
    copytree(os.path.join(src,food_item), os.path.join(dest,food_item))


In [4]:
# picking food items and generating separate data folders for the same
# food_list = ['apple_pie','chicken_quesadilla','chicken_wings', 'chocolate_cake', 'cup_cakes', 'donuts', 'french_fries', 'hamburger', 'hot_dog','ice_cream', 'macaroni_and_cheese', 'pancakes', 'pizza', 'pork_chop', 'ramen', 'red_velvet_cake', 'steak', 'tacos', 'waffles', 'omelette']
# food_list = ['apple_pie', 'chocolate_cake', 'cup_cakes', 'donuts', 'french_fries', 'hamburger', 'hot_dog','ice_cream']
food_list = ['apple_pie', 'chocolate_cake', 'cup_cakes']
dest_train = 'train_mini'
src_train = 'train'
src_test = 'test'
dest_test = 'test_mini'

In [13]:
print("Creating train data folder with new classes")
dataset_mini(food_list, src_train, dest_train)

Creating train data folder with new classes
Copying images into apple_pie
Copying images into chocolate_cake
Copying images into cup_cakes


In [14]:
print("Creating test data folder with new classes")
dataset_mini(food_list, src_test, dest_test)

Creating test data folder with new classes
Copying images into apple_pie
Copying images into chocolate_cake
Copying images into cup_cakes


Inspect the models features.

Train the model. (Does not error when running, however with the GPU currently not being detected, hold off on the first training, as it will take roughly 1:30 hour for each epoch.)

In [8]:
def make_multiclass_vgg16_model(input_shape, num_classes):
    vgg = VGG16(input_shape =input_shape, weights='imagenet', include_top=False)
    x = vgg.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128,activation='relu')(x)
    x = Dropout(0.2)(x)

    predictions = Dense(num_classes,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

    model = Model(inputs=vgg.input, outputs=predictions)
    return model

In [6]:
def make_multiclass_inception_model(num_classes):
    inception = InceptionV3(weights='imagenet', include_top=False)
    x = inception.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128,activation='relu')(x)
    x = Dropout(0.2)(x)

    predictions = Dense(num_classes,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

    model = Model(inputs=inception.input, outputs=predictions)
    return model

In [11]:
n_classes = 3
img_width, img_height = 299, 299
train_data_dir = 'train_mini'
validation_data_dir = 'test_mini'
nb_train_samples = 2250
nb_validation_samples = 750
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.175,
    zoom_range=0.175,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


FileNotFoundError: [Errno 2] No such file or directory: 'train_mini'

In [18]:
model = make_multiclass_inception_model(3)
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='best_model_3class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_3class.log')

history = model.fit(train_generator,
                    steps_per_epoch = nb_train_samples // batch_size,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,
                    epochs=30,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

Epoch 1/30
140/140 [==============================] - ETA: 0s - loss: 0.9914 - accuracy: 0.5457
Epoch 1: val_loss improved from inf to 0.72842, saving model to best_model_3class.hdf5
140/140 [==============================] - 58s 330ms/step - loss: 0.9914 - accuracy: 0.5457 - val_loss: 0.7284 - val_accuracy: 0.7717
Epoch 2/30
140/140 [==============================] - ETA: 0s - loss: 0.6885 - accuracy: 0.7753
Epoch 2: val_loss improved from 0.72842 to 0.52194, saving model to best_model_3class.hdf5
140/140 [==============================] - 30s 214ms/step - loss: 0.6885 - accuracy: 0.7753 - val_loss: 0.5219 - val_accuracy: 0.8573
Epoch 3/30
140/140 [==============================] - ETA: 0s - loss: 0.5408 - accuracy: 0.8192
Epoch 3: val_loss improved from 0.52194 to 0.41303, saving model to best_model_3class.hdf5
140/140 [==============================] - 30s 216ms/step - loss: 0.5408 - accuracy: 0.8192 - val_loss: 0.4130 - val_accuracy: 0.8804
Epoch 4/30
140/140 [=====================

In [19]:
model = make_multiclass_vgg16_model(3)
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='best_vgg16_model_3class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_3class_vgg16.log')

history = model.fit(train_generator,
                    steps_per_epoch = nb_train_samples // batch_size,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,
                    epochs=30,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

Epoch 1/30
140/140 [==============================] - ETA: 0s - loss: 1.0667 - accuracy: 0.4463
Epoch 1: val_loss improved from inf to 0.80883, saving model to best_vgg16_model_3class.hdf5
140/140 [==============================] - 51s 271ms/step - loss: 1.0667 - accuracy: 0.4463 - val_loss: 0.8088 - val_accuracy: 0.7092
Epoch 2/30
140/140 [==============================] - ETA: 0s - loss: 0.7508 - accuracy: 0.6876
Epoch 2: val_loss improved from 0.80883 to 0.53678, saving model to best_vgg16_model_3class.hdf5
140/140 [==============================] - 30s 217ms/step - loss: 0.7508 - accuracy: 0.6876 - val_loss: 0.5368 - val_accuracy: 0.8220
Epoch 3/30
140/140 [==============================] - ETA: 0s - loss: 0.5975 - accuracy: 0.7731
Epoch 3: val_loss improved from 0.53678 to 0.41782, saving model to best_vgg16_model_3class.hdf5
140/140 [==============================] - 31s 218ms/step - loss: 0.5975 - accuracy: 0.7731 - val_loss: 0.4178 - val_accuracy: 0.8723
Epoch 4/30
140/140 [===

In [39]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

ValueError: File not found: filepath=vgg16_3class_v2.keras. Please ensure the file is an accessible `.keras` zip file.

In [72]:
ml_model = ct.convert("vgg16_model", convert_to="mlprogram",inputs=[ct.ImageType(bias=[-1,-1,-1], scale=1/255)])

I0000 00:00:1732049876.693131 6074388 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1732049876.693183 6074388 single_machine.cc:361] Starting new session
I0000 00:00:1732049877.017430 6074388 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1732049877.017484 6074388 single_machine.cc:361] Starting new session
I0000 00:00:1732049877.406324 6074388 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1732049877.406372 6074388 single_machine.cc:361] Starting new session
I0000 00:00:1732049877.681953 6074388 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 

In [73]:
ml_model.save("vgg16_3class.mlpackage")

In [75]:
model.inputs

[<KerasTensor shape=(None, None, None, 3), dtype=float32, sparse=False, name=input_layer_2>]